In [8]:
import os 
import json
os.chdir("../lastfm")
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
from data import get_data
from data_processing import find_sessions
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
data = get_data()
data["timestamp"] = pd.to_datetime(data["timestamp"], errors="coerce")

In [26]:
data["hour"] = data["timestamp"].dt.hour
data.head()

,timestamp,uid,aid,hour
0,2009-05-04 23:08:57+00:00,0,0,23
1,2009-05-04 13:54:10+00:00,0,1,13
2,2009-05-04 13:52:04+00:00,0,1,13
3,2009-05-04 13:42:52+00:00,0,1,13
4,2009-05-04 13:42:11+00:00,0,1,13


## Represent listening record as a timeseries

In [45]:
user0 = data.loc[data["uid"]==0][::-1]

In [64]:
user0["diff"] = user0["timestamp"].diff().dt.seconds.fillna(0)


In [70]:
sessions = []
session = []
for i, row in user0.iterrows():
    if row["diff"]<500:
        session.append(row["aid"])
    else:
        sessions.append(session)
        session = []

In [86]:
sessions_clean = []
for session in sessions:
    session_new = []
    curr_artist = -1
    for artist in session:
        if artist != curr_artist:
            session_new.append(artist)
            curr_artist = artist
    sessions_clean.append(session_new)

### Most popular artists

In [9]:
top_artist_ids = data["aid"].value_counts()[:20]
with open("../lastfm/mappings/aid_to_artistid.json") as f:
    aid_to_artistid = json.load(f)
with open("../lastfm/mappings/artistid_to_aname.json") as f:
    artist_to_aname = json.load(f)

In [10]:
aid_to_aname = {aid: artist_to_aname.get(aid_to_artistid[aid]) for aid in aid_to_artistid}

In [16]:
top_artist_names = [aid_to_aname.get(str(aid)) for aid in top_artist_ids]
top_artist_names

[None,
 None,
 'Skeema',
 'Eddie Cano',
 'Contropotere',
 'Il Rovescio Della Medaglia',
 'Panic On The Titanic',
 'Randy Bernsen',
 'Keith Boynton',
 'Václav Neckář',
 'The Glitterboys',
 'Isaac Albéniz',
 'Paul Lowry',
 'Stars On 200',
 'In Embrace',
 'Gian Marco',
 'The Januaries',
 'Vibrasonic',
 'The Bollweevils',
 'Ranking Joe']